In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, RMSprop, Adam, Adagrad
import tensorflow as tf
import config
import utils
from model import custom_inceptionResnetV2_conv_global, custom_inceptionResnetV3_conv_global, custom_inceptionV3_attention, custom_inceptionV2_attention
import cv2
from keras.callbacks import ModelCheckpoint
import json



In [7]:

total_train_images = utils.total_count_files(config.TRAIN_DIR)
print(total_train_images)

total_val_images = utils.total_count_files(config.TESTE_DIR)
print(total_val_images)



1082
471


In [8]:
def gaussian_filter(image):
    sigmaX = 10
    return cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0,0), sigmaX), -4, 128)
     

In [9]:

dataset_train = ImageDataGenerator(
                                rescale=1/255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest',
                                preprocessing_function=gaussian_filter
                                )

train_generator = dataset_train.flow_from_directory(config.TRAIN_DIR,
                                                    classes = ["0", "1"],
                                                    target_size=config.image_size_gen,
                                                    batch_size=config.batch,
                                                    class_mode='categorical')

valida_generator = dataset_train.flow_from_directory(config.TESTE_DIR,
                                                    classes = ["0", "1"],
                                                    target_size=config.image_size_gen,
                                                    batch_size=config.batch,
                                                    class_mode='categorical')



Found 1082 images belonging to 2 classes.
Found 471 images belonging to 2 classes.


In [10]:

model_inception_v2 = custom_inceptionResnetV2_conv_global()

#optimizer
optimizer=SGD(learning_rate=1e-3,
              momentum=0.9,
              nesterov=True)
optimizer_rms=RMSprop(learning_rate=0.0001, rho=0.9, epsilon=None, decay=0.0)

optimizer_adam=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

optimizer_adagrad=Adagrad(learning_rate=0.001, epsilon=None, decay=0.0)
    
objective="categorical_crossentropy"

model_inception_v2.compile(optimizer=optimizer_adam,
              loss=objective,
              metrics=['categorical_accuracy']
              )

# callbacks
history = utils.LossHistory()
early_stopping = utils.set_early_stopping()
reduce_lr = utils.set_reduce_lr()

steps_train = int(total_train_images // config.batch)
steps_val = int(total_val_images // config.batch)


base_model loaded


MODELO INCEPTION V2 SEM ATTENTION

In [6]:
checkpoint_path = 'output/inceptionv2/normal/pesos_inception_best.h5'

checkpoint = ModelCheckpoint(checkpoint_path, 
                             monitor='categorical_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

# training model
history = model_inception_v2.fit(train_generator,
                            steps_per_epoch = steps_train,
                            epochs = config.epochs,
                            callbacks=[history, early_stopping, reduce_lr, checkpoint],
                            validation_data = valida_generator,
                            validation_steps = steps_val,
                            verbose = 2)

model_inception_v2.save_weights('output/inceptionv2/normal/pesos_inception.h5')
model_inception_v2.save('output/inceptionv2/normal/model_inception')

Epoch 1/50

Epoch 1: categorical_accuracy improved from -inf to 0.74296, saving model to output/inceptionv2/normal\pesos_inception_best.h5
67/67 - 377s - loss: 0.5371 - categorical_accuracy: 0.7430 - val_loss: 0.4578 - val_categorical_accuracy: 0.7823 - lr: 0.0010 - 377s/epoch - 6s/step
Epoch 2/50

Epoch 2: categorical_accuracy improved from 0.74296 to 0.81801, saving model to output/inceptionv2/normal\pesos_inception_best.h5
67/67 - 327s - loss: 0.4057 - categorical_accuracy: 0.8180 - val_loss: 0.4137 - val_categorical_accuracy: 0.8427 - lr: 0.0010 - 327s/epoch - 5s/step
Epoch 3/50

Epoch 3: categorical_accuracy improved from 0.81801 to 0.84803, saving model to output/inceptionv2/normal\pesos_inception_best.h5
67/67 - 302s - loss: 0.3667 - categorical_accuracy: 0.8480 - val_loss: 0.3841 - val_categorical_accuracy: 0.8427 - lr: 0.0010 - 302s/epoch - 5s/step
Epoch 4/50

Epoch 4: categorical_accuracy improved from 0.84803 to 0.85553, saving model to output/inceptionv2/normal\pesos_incept

INFO:tensorflow:Assets written to: output/inceptionv2/normal/model_inception\assets


INFO:tensorflow:Assets written to: output/inceptionv2/normal/model_inception\assets


MODE INCEPTION V3 COM ATTENTION

In [7]:
model_inception_v3_att = custom_inceptionV3_attention()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 222, 222, 3  0           []                               
                                )]                                                                
                                                                                                  
 inception_v3 (Functional)      (None, 5, 5, 2048)   21802784    ['input_3[0][0]']                
                                                                                                  
 batch_normalization_297 (Batch  (None, 5, 5, 2048)  8192        ['inception_v3[0][0]']           
 Normalization)                                                                                   
                                                                                            

In [8]:
checkpoint_path = 'output/inceptionv3/attention/pesos_inception_best.h5'

checkpoint = ModelCheckpoint(checkpoint_path, 
                             monitor='categorical_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')


history = model_inception_v3_att.fit(train_generator,
                            steps_per_epoch = steps_train,
                            epochs = config.epochs,
                            callbacks=[history, early_stopping, reduce_lr, checkpoint],
                            validation_data = valida_generator,
                            validation_steps = steps_val,
                            verbose = 2)

model_inception_v3_att.save_weights('output/inceptionv3/attention/pesos_inception.h5')
model_inception_v3_att.save('output/inceptionv3/attention/model_inception')

Epoch 1/50

Epoch 1: categorical_accuracy improved from -inf to 0.76829, saving model to output/inceptionv3/attention\pesos_inception_best.h5
67/67 - 224s - loss: 0.5362 - categorical_accuracy: 0.7683 - top_2_accuracy: 1.0000 - val_loss: 0.3965 - val_categorical_accuracy: 0.8405 - val_top_2_accuracy: 1.0000 - lr: 0.0010 - 224s/epoch - 3s/step
Epoch 2/50

Epoch 2: categorical_accuracy improved from 0.76829 to 0.83302, saving model to output/inceptionv3/attention\pesos_inception_best.h5
67/67 - 218s - loss: 0.3821 - categorical_accuracy: 0.8330 - top_2_accuracy: 1.0000 - val_loss: 0.3593 - val_categorical_accuracy: 0.8448 - val_top_2_accuracy: 1.0000 - lr: 0.0010 - 218s/epoch - 3s/step
Epoch 3/50

Epoch 3: categorical_accuracy improved from 0.83302 to 0.84053, saving model to output/inceptionv3/attention\pesos_inception_best.h5
67/67 - 217s - loss: 0.3690 - categorical_accuracy: 0.8405 - top_2_accuracy: 1.0000 - val_loss: 0.3430 - val_categorical_accuracy: 0.8470 - val_top_2_accuracy: 1.

INFO:tensorflow:Assets written to: output/inceptionv3/attention/model_inception\assets


INFO:tensorflow:Assets written to: output/inceptionv3/attention/model_inception\assets


MODELO INCEPTION V3 SEM ATTENTION

In [13]:
model_inception_v3_normal = custom_inceptionResnetV3_conv_global()

model_inception_v3_normal.compile(optimizer=optimizer_adam,
              loss=objective,
              metrics=['categorical_accuracy']
              )

base_model loaded


In [14]:

checkpoint_path = 'output/inceptionv3/normal/pesos_inception_best.h5'

checkpoint = ModelCheckpoint(checkpoint_path, 
                             monitor='categorical_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

history = model_inception_v3_normal.fit(train_generator,
                            steps_per_epoch = steps_train,
                            epochs = config.epochs,
                            callbacks=[history, early_stopping, reduce_lr, checkpoint],
                            validation_data = valida_generator,
                            validation_steps = steps_val,
                            verbose = 2)

model_inception_v3_normal.save_weights('output/inceptionv3/normal/pesos_inception.h5')
model_inception_v3_normal.save('output/inceptionv3/normal/model_inception')

Epoch 1/50

Epoch 1: categorical_accuracy improved from -inf to 0.73452, saving model to output/inceptionv3/normal\pesos_inception_best.h5
67/67 - 243s - loss: 0.5628 - categorical_accuracy: 0.7345 - val_loss: 0.4647 - val_categorical_accuracy: 0.7931 - lr: 0.0010 - 243s/epoch - 4s/step
Epoch 2/50

Epoch 2: categorical_accuracy improved from 0.73452 to 0.83677, saving model to output/inceptionv3/normal\pesos_inception_best.h5
67/67 - 232s - loss: 0.3938 - categorical_accuracy: 0.8368 - val_loss: 0.4296 - val_categorical_accuracy: 0.8233 - lr: 0.0010 - 232s/epoch - 3s/step
Epoch 3/50

Epoch 3: categorical_accuracy improved from 0.83677 to 0.84146, saving model to output/inceptionv3/normal\pesos_inception_best.h5
67/67 - 232s - loss: 0.3513 - categorical_accuracy: 0.8415 - val_loss: 0.4254 - val_categorical_accuracy: 0.8254 - lr: 0.0010 - 232s/epoch - 3s/step
Epoch 4/50

Epoch 4: categorical_accuracy improved from 0.84146 to 0.84522, saving model to output/inceptionv3/normal\pesos_incept

INFO:tensorflow:Assets written to: output/inceptionv3/normal/model_inception\assets


INFO:tensorflow:Assets written to: output/inceptionv3/normal/model_inception\assets


In [20]:
history.history

{'loss': [0.5628063678741455,
  0.39380645751953125,
  0.35126349329948425,
  0.35893043875694275,
  0.3610437214374542,
  0.3304060399532318,
  0.3436901867389679,
  0.33589157462120056,
  0.33416199684143066,
  0.29829707741737366,
  0.34057098627090454,
  0.3118400573730469,
  0.3127361834049225,
  0.2893204092979431,
  0.307861328125,
  0.3109379708766937],
 'categorical_accuracy': [0.7345215678215027,
  0.8367729783058167,
  0.8414633870124817,
  0.8452157378196716,
  0.8452157378196716,
  0.8630393743515015,
  0.8499062061309814,
  0.8527204394340515,
  0.8621013164520264,
  0.8752345442771912,
  0.8527204394340515,
  0.8630393743515015,
  0.8602251410484314,
  0.8658536672592163,
  0.8696060180664062,
  0.8677298426628113],
 'val_loss': [0.46467846632003784,
  0.4296267330646515,
  0.42544519901275635,
  0.39915329217910767,
  0.3711182475090027,
  0.4049389064311981,
  0.38298866152763367,
  0.4177466034889221,
  0.37070199847221375,
  0.3858361840248108,
  0.361804336309433,
 

In [22]:

with open('./output/inceptionv3/normal/history_v3_normal.json', 'w') as h:
    json.dump(str(history.history), h)

In [23]:
modelo_inception_v2_attention = custom_inceptionV2_attention()

checkpoint_path = 'output/inceptionv2/attention/pesos_inception_best.h5'

checkpoint = ModelCheckpoint(checkpoint_path, 
                             monitor='categorical_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

base_model loaded
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 222, 222, 3  0           []                               
                                )]                                                                
                                                                                                  
 inception_resnet_v2 (Functiona  (None, 5, 5, 1536)  54336736    ['input_8[0][0]']                
 l)                                                                                               
                                                                                                  
 batch_normalization_703 (Batch  (None, 5, 5, 1536)  6144        ['inception_resnet_v2[1][0]']    
 Normalization)                                                           

In [24]:

history = modelo_inception_v2_attention.fit(train_generator,
                            steps_per_epoch = steps_train,
                            epochs = config.epochs,
                            callbacks=[history, reduce_lr, checkpoint],
                            validation_data = valida_generator,
                            validation_steps = steps_val,
                            verbose = 2)

modelo_inception_v2_attention.save_weights('output/inceptionv2/attention/pesos_inception.h5')
modelo_inception_v2_attention.save('output/inceptionv2/attention/model_inception')

with open('./output/inceptionv2/attention/history_v2_attention.json', 'w') as h:
    json.dump(str(history.history), h)

Epoch 1/50

Epoch 1: categorical_accuracy improved from -inf to 0.78424, saving model to output/inceptionv2/attention\pesos_inception_best.h5
67/67 - 376s - loss: 0.4968 - categorical_accuracy: 0.7842 - top_2_accuracy: 1.0000 - val_loss: 0.4708 - val_categorical_accuracy: 0.7909 - val_top_2_accuracy: 1.0000 - lr: 0.0010 - 376s/epoch - 6s/step
Epoch 2/50

Epoch 2: categorical_accuracy improved from 0.78424 to 0.81707, saving model to output/inceptionv2/attention\pesos_inception_best.h5
67/67 - 351s - loss: 0.3936 - categorical_accuracy: 0.8171 - top_2_accuracy: 1.0000 - val_loss: 0.5300 - val_categorical_accuracy: 0.7737 - val_top_2_accuracy: 1.0000 - lr: 0.0010 - 351s/epoch - 5s/step
Epoch 3/50

Epoch 3: categorical_accuracy improved from 0.81707 to 0.84428, saving model to output/inceptionv2/attention\pesos_inception_best.h5
67/67 - 359s - loss: 0.3342 - categorical_accuracy: 0.8443 - top_2_accuracy: 1.0000 - val_loss: 0.3637 - val_categorical_accuracy: 0.8556 - val_top_2_accuracy: 1.

INFO:tensorflow:Assets written to: output/inceptionv2/attention/model_inception\assets


INFO:tensorflow:Assets written to: output/inceptionv2/attention/model_inception\assets


In [25]:
modelo_inception_v2_attention_t = modelo_inception_v2_attention

In [ ]:
modelo_inception_v2_attention_t.load_weights('output/inceptionv2/attention/pesos_inception_best.h5')

In [ ]:

image = utils.load('./datasets/VALIDACAO/images/training/IDRiD_197.jpg')
model.evaluate(valida_generator)


In [ ]:
# training model
history = model.fit(train_generator,
                            steps_per_epoch = steps_train,
                            epochs = 5, #config.epochs,
                            callbacks=[history, early_stopping, reduce_lr],
                            validation_data = valida_generator,
                            validation_steps = steps_val,
                            verbose = 2)